
# Import and model

In [14]:
from torchvision.transforms import Compose, Resize, Normalize, ToTensor
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import numpy as np
import torch
import pickle
import os
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32')
processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')

transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
])

c:\Users\DELL\miniconda3\envs\CS336\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
def normalize_data(data):
    # Kiểm tra nếu data là một vector 1 chiều
    if data.ndim == 1:
        normalized_data = data / np.linalg.norm(data)
    else:
        # Chuẩn hóa dữ liệu nếu là mảng 2 chiều
        normalized_data = data / np.linalg.norm(data, axis=1, keepdims=True)
    
    return normalized_data

In [17]:
def extract_clip_feature(images, device='cpu'):
#   images = [Image.open(path).convert('RGB') for path in image_paths]
  features = []
  for image in images:
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        feature = model.get_image_features(image)
    feature = feature.squeeze().cpu().numpy()
    features.append(feature)

  return features

# Parameters

In [18]:
gt_path = r'C:\Retrieval System\data\paris_120310_gt'
data_path = r'C:\Retrieval System\data\paris'
features_path = r'C:\Retrieval System\features\features_paris_clip'

In [19]:
k = 5

# Get input features

In [20]:
def process_queries(query_file, device='cpu'):
    """
    Đọc file query, cắt ảnh theo bounding box và extract feature theo từng bb.
    
    :param query_file: Đường dẫn đến file query.txt
    :param image_folder: Thư mục chứa ảnh
    """
    with open(query_file, 'r') as file:
        features = []
        for line in file:
            # Tách dòng query thành các phần tử
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            
            image_name = parts[0] + '.jpg'  # Tên file ảnh
            x1, y1, x2, y2 = map(float, parts[1:])  # Bounding box
            
            # Đường dẫn đầy đủ đến ảnh
            image_path = os.path.join(data_path, image_name.split('_')[1], image_name)
            
            # Kiểm tra ảnh tồn tại
            if not os.path.exists(image_path):
                print(f"Ảnh {image_name} không tồn tại trong thư mục {os.path.join(data_path, image_name.split('_')[1])}.")
                continue
            
            # Mở ảnh và cắt theo bounding box
            with Image.open(image_path).convert('RGB') as img:
                cropped_img = img.crop((x1, y1, x2, y2))  # Cắt ảnh
                # Extract feature của ảnh crop
                cropped_img = transform(cropped_img).unsqueeze(0).to(device)
                with torch.no_grad():
                    feature = model.get_image_features(cropped_img)
                feature = feature.squeeze().cpu().numpy()
                features.append(feature)
        
        return features

In [21]:
query = []
for file in os.listdir(gt_path):
    if 'query' in file:
        print(file)
        query_file = os.path.join(gt_path, file)
        features = process_queries(query_file=query_file)
        if len(features) == 1:
            query.append(features[0])

input_features = np.array(query)
input_features.shape

defense_1_query.txt
defense_2_query.txt
defense_3_query.txt
defense_4_query.txt
defense_5_query.txt
eiffel_1_query.txt
eiffel_2_query.txt
eiffel_3_query.txt
eiffel_4_query.txt
eiffel_5_query.txt
invalides_1_query.txt
invalides_2_query.txt
invalides_3_query.txt
invalides_4_query.txt
invalides_5_query.txt
louvre_1_query.txt
louvre_2_query.txt
louvre_3_query.txt
louvre_4_query.txt
louvre_5_query.txt
moulinrouge_1_query.txt
moulinrouge_2_query.txt
moulinrouge_3_query.txt
moulinrouge_4_query.txt
moulinrouge_5_query.txt
museedorsay_1_query.txt
museedorsay_2_query.txt
museedorsay_3_query.txt
museedorsay_4_query.txt
museedorsay_5_query.txt
notredame_1_query.txt
notredame_2_query.txt
notredame_3_query.txt
notredame_4_query.txt
notredame_5_query.txt
pantheon_1_query.txt
pantheon_2_query.txt
pantheon_3_query.txt
pantheon_4_query.txt
pantheon_5_query.txt
pompidou_1_query.txt
pompidou_2_query.txt
pompidou_3_query.txt
pompidou_4_query.txt
pompidou_5_query.txt
sacrecoeur_1_query.txt
sacrecoeur_2_quer

(55, 512)

# Load database

In [22]:
database = []
map_db = []
for file in os.listdir(features_path):
    file_path = os.path.join(features_path, file)

    with open(file_path, "rb") as f:
        image_features_data = pickle.load(f) # 1 list, các phần tử là dict với image_name và features

        for item in image_features_data:
            database.append(item['features'])
            map_db.append(item['image_name'])

database = np.array(database)
database.shape

(6392, 512)

# Retrieval

In [54]:
similarity_matrix = cosine_similarity(query, database)
similarity_matrix[0]

array([0.7190387 , 0.7383275 , 0.7878091 , ..., 0.67330176, 0.6710704 ,
       0.6382113 ], dtype=float32)

In [55]:
sorted_similarity_indices = np.argsort(similarity_matrix, axis=1)[:, ::-1]
print(sorted_similarity_indices[0])

# Sắp xếp lại map_db theo thứ tự của similarity_matrix
sorted_map_db = []
for i in range(similarity_matrix.shape[0]):
    sorted_map_db.append([map_db[j] for j in sorted_similarity_indices[i]])

[  57  168   78 ... 3410 3743 3903]
